# packages and db

In [160]:
import pymongo
import pandas as pd
import datetime

myclient = pymongo.MongoClient('mongodb://localhost:27017')
scrapedb = myclient['scrapedb']
adcollection = scrapedb['ads']
adcollection_d = scrapedb['ads_deduplicated']

pd.set_option("display.max_rows", None, "display.max_columns", None)

# precalculations

In [161]:
# vary basic handling of outliers
# price
adcollection.update_many(
	{'$or':[{'properties.Cena':{'$lt':30000}}, {'properties.Cena':{'$gt':900000}}]},
	{'$set':{'properties.Cena':170000}},
	upsert=False
)

# price m2
adcollection.update_many(
	{'properties.Cena za meter':{'$gt':10000}},
	{'$set':{'properties.Cena za meter':3000}},
	upsert=False
)

# size
adcollection.update_many(
	{'properties.Úžitková plocha':{'$lt':10}},
	{'$set':{'properties.Úžitková plocha':40}},
	upsert=False
)


# refresh collection without duplicates - take only the last one if scraped multiple times
adcollection.aggregate([
	{'$sort':{'scraped timestamp':1}},
	{'$group':{
		'_id':'$properties.link',
		'ad':{'$last':'$$ROOT'}
		}
	},
	{'$replaceRoot':{'newRoot':'$ad'}},
	{'$out':'ads_deduplicated'}

	])

locs = ['Bratislava I, Staré Mesto',
'Bratislava II, Ružinov',
'Bratislava V, Petržalka',
'Bratislava III, Nové Mesto',
'Bratislava IV, Dúbravka',
'Bratislava IV, Karlova Ves',
'Bratislava II, Vrakuňa',
'Bratislava III, Rača',
'Bratislava II, Podunajské Biskupice',
'Bratislava IV, Devínska Nová Ves',
'Bratislava V, Jarovce',
'Bratislava IV, Lamač']

# date filter for overall stats

In [162]:
start = datetime.datetime(2020, 5, 10, 0, 0, 0)
end = datetime.datetime.now()

In [163]:
q0 = list(adcollection.aggregate([
	{'$group':{'_id':'global',
			'scraped_date_min':{'$min':'$scraped timestamp'}
		}
  
  }
	
]))[0]['scraped_date_min']
	
print(f"First item scraped: {q0}")

First item scraped: 2020-05-10 13:28:57.405000


In [164]:
q0 = list(adcollection.aggregate([
	{'$group':{'_id':'global',
			'scraped_date_max':{'$max':'$scraped timestamp'}
		}
  
  }
	
]))[0]['scraped_date_max']
	
print(f"Last item scraped: {q0}")

Last item scraped: 2020-06-21 17:17:51.680000


# COUNTS

## number of all documents

In [165]:
q1 = adcollection.count_documents({'scraped timestamp': {'$lt': end, '$gte': start}})
print(f"Number of all documents: {q1}")

Number of all documents: 7795


## number of scraped documents per week

In [166]:
q2 = list(adcollection.aggregate([
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                                       {'$toString':{'$week':'$scraped timestamp'}}]
                            }
                }
    },
    {'$group':{'_id':'$yearweek', 'scraped_documents_count':{'$sum':1}}}
    
    
]))

df2 = pd.json_normalize(q2)\
    .rename(columns = {'_id':'yearweek'})\
    .sort_values('yearweek')
df2

,yearweek,scraped_documents_count
2,202019,312
4,202020,305
1,202021,800
3,202022,1594
6,202023,1597
0,202024,1595
5,202025,1592


## number of distinct ads

In [167]:
q3 = len(adcollection.distinct('properties.link'))
print(f"Number of distinct ads: {q3}")

Number of distinct ads: 3849


## number of ads with tags

In [168]:
q4 = adcollection_d.count_documents({'tags.1':{'$exists':True}, 'scraped timestamp': {'$lt': end, '$gte': start}})
print(f"Number of ads with atleast 1 tag: {q4}")

Number of ads with atleast 1 tag: 1737


In [216]:
q4_0 = list(adcollection.aggregate([    
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                                       {'$toString':{'$week':'$scraped timestamp'}}]
                            },
                'tags':1}
    },
    {'$match':{'tags.1':{'$exists':True}}},
    {'$group':{'_id':'$yearweek', 'documents_count':{'$sum':1}}}
    
    
]))

df4_0 = pd.json_normalize(q4_0)\
    .rename(columns = {'_id':'yearweek', 'documents_count':'ads_with_tags_count'})\
    .sort_values('yearweek')
df4_0.pivot_table(columns = 'yearweek')

yearweek,202019,202020,202021,202022,202023,202024,202025
ads_with_tags_count,121,109,344,689,746,744,700


## counts of tags

In [217]:
q4_1 = list(adcollection_d.aggregate([
    {'$match': {'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$unwind':'$tags'},
	{'$group':{'_id':'$tags', 'tag_count':{'$sum':1}
		}
  
  }
	
]))
	
df4_1 = pd.json_normalize(q4_1)\
    .rename(columns = {'_id':'tag'})\
    .sort_values('tag_count', ascending = False)
df4_1.head(20)

,tag,tag_count
7,Výťah,1290
54,Blízkosť zastávky MHD,1078
65,Nákupné centrum v blízkosti,1070
27,Plastové okná,1050
46,Škola v blízkosti,920
83,Tichá lokalita,891
68,Zateplený bytový dom,775
55,Nádherný výhľad,721
86,loggia,666
2,Balkón,662


In [229]:
q4_2 = list(adcollection.aggregate([
    {'$match': {'scraped timestamp': {'$lt': end, '$gte': start}}},
        {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                                       {'$toString':{'$week':'$scraped timestamp'}}]
                            },
                'tags':1}
    },
	{'$unwind':'$tags'},
	{'$group':{'_id':{'yearweek':'$yearweek', 'tags':'$tags'}, 'tag_count':{'$sum':1}
		}
  
  }
	
]))
	
df4_2 = pd.json_normalize(q4_2)\
    .rename(columns = {'_id.tags':'tag', '_id.yearweek':'yearweek'})\
    .sort_values('tag_count', ascending = False)
df4_2 = df4_2.pivot_table(columns = 'yearweek', index = 'tag', values = 'tag_count')
df4_2['total'] = df4_2.sum(axis = 1)
df4_2 = df4_2.sort_values(by = 'total', ascending = False)
df4_2.head(20)

yearweek,202019,202020,202021,202022,202023,202024,202025,total
tag,,,,,,,,
Výťah,89.0,80.0,252.0,545.0,581.0,572.0,540.0,2659.0
Blízkosť zastávky MHD,69.0,62.0,191.0,391.0,436.0,391.0,423.0,1963.0
Nákupné centrum v blízkosti,78.0,65.0,194.0,385.0,429.0,389.0,414.0,1954.0
Plastové okná,71.0,58.0,194.0,388.0,439.0,402.0,396.0,1948.0
Balkón,39.0,50.0,149.0,324.0,325.0,313.0,301.0,1501.0
Tichá lokalita,71.0,49.0,154.0,278.0,333.0,294.0,289.0,1468.0
Škola v blízkosti,70.0,54.0,146.0,287.0,323.0,287.0,296.0,1463.0
Zateplený bytový dom,56.0,41.0,133.0,259.0,305.0,265.0,264.0,1323.0
Nádherný výhľad,57.0,37.0,120.0,237.0,281.0,217.0,246.0,1195.0


## number of ads with map coordinates

In [170]:
q5 = adcollection_d.count_documents({'mapcoord.lon':{'$exists':True}, 'scraped timestamp': {'$lt': end, '$gte': start}})
print(f"Number of ads with map coordinates: {q5}")

Number of ads with map coordinates: 3399


In [230]:
q5_1 = list(adcollection.aggregate([
    {'$match': {'scraped timestamp': {'$lt': end, '$gte': start}}},
        {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                                       {'$toString':{'$week':'$scraped timestamp'}}]
                            },
                'mapcoord':1}
    },
	{'$group':{'_id':'$yearweek', 'with_mapcoord_count':{'$sum':1}
		}
  
  }
	
]))
	
df5_1 = pd.json_normalize(q5_1)\
    .rename(columns = {'_id':'yearweek'})
df5_1 = df5_1.pivot_table(columns = 'yearweek')
df5_1['total'] = df5_1.sum(axis = 1)
df5_1

yearweek,202019,202020,202021,202022,202023,202024,202025,total
with_mapcoord_count,312,305,800,1594,1597,1595,1592,7795


## counts of keys in properties

In [171]:
q5_1 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$project':{
		'properties':{'$objectToArray':'$properties'}
		}
  
  },
	{'$unwind':'$properties'},
	{'$group':{'_id':'$properties.k',
			'ads_count':{'$sum':1}
		}
  
  }
	
]))
	
df5_1 = pd.json_normalize(q5_1)\
    .rename(columns = {'_id':'property'})\
    .sort_values('ads_count', ascending = False)
df5_1

,property,ads_count
11,Aktualizácia,3849
3,Kategória,3849
16,Pivnica,3849
8,Lokalita,3849
15,Cena dohodou,3849
1,Balkón / loggia,3849
13,link,3849
12,Úžitková plocha,3830
19,Cena,3735
17,Provízia v cene,3735


## counts of distinct values in properties

In [172]:
q5_2 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$project':{
		'properties':{'$objectToArray':'$properties'}
		}
  
  },
	{'$unwind':'$properties'},
	{'$group':{'_id':'$properties.k',
			'unique_values':{'$addToSet': "$properties.v"}
		}
    },
     {'$project':{'_id':1,
                 'unique_values_count':{'$size':'$unique_values'}}
     
     }
  

	
]))
	
df5_2 = pd.json_normalize(q5_2)\
    .rename(columns = {'_id':'property'})\
    .sort_values('unique_values_count', ascending = False)
df5_2

,property,unique_values_count
11,link,3849
18,Identifikačné číslo:,2840
14,Cena za meter,1559
9,Aktualizácia,1548
6,Cena,994
10,Ulica,738
7,Úžitková plocha,210
16,Lokalita,65
4,Zastavaná plocha,54
13,Pozemok m2,47


## number of ads per flat category

In [235]:
q6_1 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$group':{'_id':'$properties.Kategória', 'ads_count':{'$sum':1}}}]))

df6_1 = pd.json_normalize(q6_1)\
    .rename(columns = {'_id':'flat_category'})\
    .sort_values('ads_count', ascending = False)
df6_1

,flat_category,ads_count
7,2 izbový byt,2657
4,3 izbový byt,2561
0,4 izbový byt,1070
2,1 izbový byt,778
6,Garsónka,336
3,5 izbový byt a viac,210
1,Dvojgarsónka,110
5,Mezonet,33
9,Apartmán,23
8,Iný byt,17


In [239]:
q6 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                               {'$toString':{'$week':'$scraped timestamp'}}]
                    },
        'properties':1}
    },
    {'$group':{'_id':{'yearweek':'$yearweek', 'flat_category':'$properties.Kategória'}, 'ads_count':{'$sum':1}}}]))

df6 = pd.json_normalize(q6)\
    .rename(columns = {'_id.yearweek':'yearweek', '_id.flat_category':'flat_category'})\
    .sort_values('ads_count', ascending = False)
df6 = df6.pivot_table(columns = 'yearweek', index = 'flat_category', values = 'ads_count')
df6['total'] = df6.sum(axis = 1)
df6

yearweek,202019,202020,202021,202022,202023,202024,202025,total
flat_category,,,,,,,,
1 izbový byt,35.0,34.0,77.0,153.0,164.0,166.0,149.0,778.0
2 izbový byt,112.0,104.0,269.0,540.0,555.0,539.0,538.0,2657.0
3 izbový byt,101.0,96.0,247.0,508.0,525.0,532.0,552.0,2561.0
4 izbový byt,45.0,46.0,132.0,230.0,210.0,208.0,199.0,1070.0
5 izbový byt a viac,5.0,7.0,27.0,58.0,37.0,32.0,44.0,210.0
Apartmán,1.0,2.0,1.0,4.0,3.0,6.0,6.0,23.0
Dvojgarsónka,3.0,1.0,11.0,22.0,24.0,26.0,23.0,110.0
Garsónka,8.0,12.0,29.0,72.0,67.0,78.0,70.0,336.0
Iný byt,NaN,1.0,3.0,2.0,4.0,3.0,4.0,17.0


## number of ads per state and category

In [240]:
q8 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$group':{'_id':{'kategoria':'$properties.Kategória', 'stav':'$properties.Stav'}, 'ads_count':{'$sum':1}}}]))

df8 = pd.json_normalize(q8)\
    .rename(columns = {'_id.kategoria':'category', '_id.stav':'state'})\
    .sort_values(['state', 'ads_count'], ascending = False)
df8 = df8[['state', 'category', 'ads_count']]
df8 = df8.pivot(index='state', columns='category', values='ads_count')
df8['state_count'] = df8.sum(axis = 1)
df8 = df8.sort_values('state_count', ascending = False)
df8

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet,state_count
state,,,,,,,,,,,
kompletná rekonštrukcia,124.0,570.0,543.0,181.0,17.0,1.0,34.0,65.0,2.0,6.0,1543.0
NaN,136.0,458.0,282.0,158.0,49.0,6.0,4.0,32.0,2.0,4.0,1131.0
čiastočná rekonštrukcia,85.0,187.0,272.0,111.0,13.0,NaN,15.0,48.0,NaN,1.0,732.0
pôvodný,33.0,74.0,127.0,33.0,9.0,1.0,2.0,36.0,NaN,NaN,315.0
vo výstavbe,14.0,54.0,33.0,16.0,5.0,1.0,NaN,1.0,2.0,2.0,128.0


In [249]:
q8 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                           {'$toString':{'$week':'$scraped timestamp'}}]
                },
    'properties':1}
    },
    {'$group':{'_id':{'yearweek':'$yearweek', 'kategoria':'$properties.Kategória', 'stav':'$properties.Stav'}, 'ads_count':{'$sum':1}}}]))

df8 = pd.json_normalize(q8)\
    .rename(columns = {'_id.kategoria':'category', '_id.stav':'state', '_id.yearweek':'yearweek'})\
    .sort_values(['state', 'ads_count'], ascending = False)
df8 = df8[['state', 'category', 'yearweek','ads_count']]
df8 = df8.pivot_table(index=['category','state'], columns='yearweek', values='ads_count')
df8['total'] = df8.sum(axis = 1)

df8

yearweek                                     202019  202020  202021  202022  \
category            state                                                     
1 izbový byt        kompletná rekonštrukcia     7.0    15.0    22.0    40.0   
                    pôvodný                     4.0     5.0    11.0    17.0   
                    vo výstavbe                 1.0     1.0     5.0     9.0   
                    čiastočná rekonštrukcia     7.0     5.0     8.0    22.0   
2 izbový byt        kompletná rekonštrukcia    45.0    38.0   113.0   210.0   
                    pôvodný                     3.0     5.0    14.0    30.0   
                    vo výstavbe                 3.0     5.0    21.0    37.0   
                    čiastočná rekonštrukcia    15.0    17.0    33.0    65.0   
3 izbový byt        kompletná rekonštrukcia    31.0    37.0   123.0   223.0   
                    pôvodný                     7.0     9.0    19.0    39.0   
                    vo výstavbe                 6.0     2.0    11.0    23.0   
                    čiastočná rekonštrukcia    18.0    16.0    35.0   100.0   
4 izbový byt        kompletná rekonštrukcia    12.0    17.0    50.0    72.0   
                    pôvodný                     5.0     3.0    12.0    14.0   
                    vo výstavbe                 1.0     2.0     2.0    13.0   
                    čiastočná rekonštrukcia    10.0     8.0    27.0    42.0   
5 izbový byt a viac kompletná rekonštrukcia     NaN     NaN     3.0     9.0   
                    pôvodný                     NaN     1.0     3.0     5.0   
                    vo výstavbe                 1.0     1.0     1.0     4.0   
                    čiastočná rekonštrukcia     NaN     1.0     3.0     6.0   
Apartmán            kompletná rekonštrukcia     NaN     NaN     NaN     NaN   
                    pôvodný                     NaN     NaN     1.0     1.0   
                    vo výstavbe                 NaN     1.0     NaN     NaN   
Dvojgarsónka        kompletná rekonštrukcia     1.0     1.0     2.0     9.0   
                    pôvodný                     1.0     NaN     1.0     2.0   
                    čiastočná rekonštrukcia     1.0     NaN     8.0    10.0   
Garsónka            kompletná rekonštrukcia     2.0     2.0    13.0    29.0   
                    pôvodný                     2.0     2.0     5.0    15.0   
                    vo výstavbe                 NaN     NaN     NaN     1.0   
                    čiastočná rekonštrukcia     3.0     4.0     9.0    18.0   
Iný byt             kompletná rekonštrukcia     NaN     NaN     2.0     NaN   
                    vo výstavbe                 NaN     NaN     1.0     2.0   
Mezonet             kompletná rekonštrukcia     NaN     1.0     1.0     2.0   
                    vo výstavbe                 1.0     1.0     1.0     1.0   
                    čiastočná rekonštrukcia     NaN     NaN     NaN     NaN   

yearweek                                     202023  202024  202025   total  
category            state                                                    
1 izbový byt        kompletná rekonštrukcia    45.0    49.0    51.0   229.0  
                    pôvodný                    17.0    12.0    11.0    77.0  
                    vo výstavbe                 7.0    10.0     9.0    42.0  
                    čiastočná rekonštrukcia    30.0    36.0    31.0   139.0  
2 izbový byt        kompletná rekonštrukcia   237.0   226.0   228.0  1097.0  
                    pôvodný                    26.0    27.0    25.0   130.0  
                    vo výstavbe                40.0    33.0    36.0   175.0  
                    čiastočná rekonštrukcia    68.0    76.0    73.0   347.0  
3 izbový byt        kompletná rekonštrukcia   239.0   229.0   253.0  1135.0  
                    pôvodný                    46.0    52.0    50.0   222.0  
                    vo výstavbe                22.0    20.0    21.0   105.0  
                    čiastočná rekonštrukcia   110.0   121.0   

## number of ads per location and category

### counts

In [175]:
q10 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$group':{'_id':{'kategoria':'$properties.Kategória', 'lokalita':'$properties.Lokalita'}, 'ads_count':{'$sum':1}}}]))

df10 = pd.json_normalize(q10)\
    .rename(columns = {'_id.kategoria':'category', '_id.lokalita':'location'})
df10 = df10.pivot(index='location', columns='category', values='ads_count')
df10['location_count'] = df10.sum(axis = 1)
df10 = df10.sort_values('location_count', ascending = False)
df10.head(30)

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet,location_count
location,,,,,,,,,,,
"Bratislava V, Petržalka",54.0,188.0,296.0,62.0,9.0,1.0,41.0,45.0,1.0,1.0,698.0
"Bratislava I, Staré Mesto",49.0,244.0,238.0,90.0,33.0,4.0,NaN,20.0,2.0,4.0,684.0
"Bratislava II, Ružinov",83.0,291.0,185.0,57.0,1.0,1.0,4.0,48.0,1.0,4.0,675.0
"Bratislava III, Nové Mesto",29.0,158.0,109.0,50.0,10.0,2.0,NaN,9.0,2.0,1.0,370.0
"Bratislava IV, Karlova Ves",49.0,47.0,89.0,33.0,11.0,NaN,NaN,NaN,NaN,NaN,229.0
"Bratislava IV, Dúbravka",20.0,69.0,59.0,48.0,3.0,NaN,1.0,23.0,NaN,1.0,224.0
"Bratislava III, Rača",24.0,77.0,30.0,16.0,4.0,NaN,NaN,5.0,NaN,NaN,156.0
"Bratislava II, Vrakuňa",7.0,33.0,38.0,51.0,5.0,NaN,7.0,4.0,NaN,NaN,145.0
"Bratislava II, Podunajské Biskupice",14.0,47.0,55.0,16.0,3.0,NaN,NaN,3.0,NaN,NaN,138.0


### percent

In [176]:
df11 = df10.iloc[:,[i for i in range(0, 10)]].apply(lambda x: x/df10['location_count'])
df11.head(30)

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet
location,,,,,,,,,,
"Bratislava V, Petržalka",0.077364,0.269341,0.424069,0.088825,0.012894,0.001433,0.058739,0.064470,0.001433,0.001433
"Bratislava I, Staré Mesto",0.071637,0.356725,0.347953,0.131579,0.048246,0.005848,NaN,0.029240,0.002924,0.005848
"Bratislava II, Ružinov",0.122963,0.431111,0.274074,0.084444,0.001481,0.001481,0.005926,0.071111,0.001481,0.005926
"Bratislava III, Nové Mesto",0.078378,0.427027,0.294595,0.135135,0.027027,0.005405,NaN,0.024324,0.005405,0.002703
"Bratislava IV, Karlova Ves",0.213974,0.205240,0.388646,0.144105,0.048035,NaN,NaN,NaN,NaN,NaN
"Bratislava IV, Dúbravka",0.089286,0.308036,0.263393,0.214286,0.013393,NaN,0.004464,0.102679,NaN,0.004464
"Bratislava III, Rača",0.153846,0.493590,0.192308,0.102564,0.025641,NaN,NaN,0.032051,NaN,NaN
"Bratislava II, Vrakuňa",0.048276,0.227586,0.262069,0.351724,0.034483,NaN,0.048276,0.027586,NaN,NaN
"Bratislava II, Podunajské Biskupice",0.101449,0.340580,0.398551,0.115942,0.021739,NaN,NaN,0.021739,NaN,NaN


### mean percent of category in top few locations

In [177]:
df11.loc[locs,:].mean()


category
1 izbový byt           0.131355
2 izbový byt           0.370793
3 izbový byt           0.275824
4 izbový byt           0.165620
5 izbový byt a viac    0.031189
Apartmán               0.003542
Dvojgarsónka           0.029351
Garsónka               0.047248
Iný byt                0.002811
Mezonet                0.007782
dtype: float64

## counts per yearweek

In [253]:
q8 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                           {'$toString':{'$week':'$scraped timestamp'}}]
                },
    'properties':1}
    },
    {'$group':{'_id':{'yearweek':'$yearweek', 'kategoria':'$properties.Kategória', 'location':'$properties.Lokalita'}, 'ads_count':{'$sum':1}}}]))

df8 = pd.json_normalize(q8)\
    .rename(columns = {'_id.kategoria':'category', '_id.location':'location', '_id.yearweek':'yearweek'})
df8 = df8[['location', 'category', 'yearweek','ads_count']]
df8 = df8.loc[df8['location'].isin(locs),:]
df8 = df8.pivot_table(index=['category','location'], columns='yearweek', values='ads_count')
df8['total'] = df8.sum(axis = 1)

df8

yearweek                                                 202019  202020  \
category            location                                              
1 izbový byt        Bratislava I, Staré Mesto               3.0     4.0   
                    Bratislava II, Podunajské Biskupice     NaN     NaN   
                    Bratislava II, Ružinov                  4.0     8.0   
                    Bratislava II, Vrakuňa                  NaN     NaN   
                    Bratislava III, Nové Mesto              3.0     3.0   
                    Bratislava III, Rača                    3.0     3.0   
                    Bratislava IV, Devínska Nová Ves        3.0     2.0   
                    Bratislava IV, Dúbravka                 1.0     2.0   
                    Bratislava IV, Karlova Ves              NaN     NaN   
                    Bratislava IV, Lamač                    NaN     NaN   
                    Bratislava V, Petržalka                 6.0     6.0   
2 izbový byt        Bratislava I, Staré Mesto              21.0    24.0   
                    Bratislava II, Podunajské Biskupice     4.0     NaN   
                    Bratislava II, Ružinov                 18.0    19.0   
                    Bratislava II, Vrakuňa                  2.0     4.0   
                    Bratislava III, Nové Mesto             12.0    14.0   
                    Bratislava III, Rača                    3.0     5.0   
                    Bratislava IV, Devínska Nová Ves        2.0     2.0   
                    Bratislava IV, Dúbravka                 8.0     9.0   
                    Bratislava IV, Karlova Ves              2.0     3.0   
                    Bratislava IV, Lamač                    1.0     1.0   
                    Bratislava V, Jarovce                   NaN     3.0   
                    Bratislava V, Petržalka                 9.0    14.0   
3 izbový byt        Bratislava I, Staré Mesto               9.0    21.0   
                    Bratislava II, Podunajské Biskupice     3.0     6.0   
                    Bratislava II, Ružinov                 13.0    17.0   
                    Bratislava II, Vrakuňa                  NaN     5.0   
                    Bratislava III, Nové Mesto              8.0     8.0   
                    Bratislava III, Rača                    1.0     1.0   
                    Bratislava IV, Devínska Nová Ves        4.0     2.0   
                    Bratislava IV, Dúbravka                 6.0     3.0   
                    Bratislava IV, Karlova Ves              5.0     4.0   
                    Bratislava IV, Lamač                    1.0     NaN   
                    Bratislava V, Jarovce                   NaN     NaN   
                    Bratislava V, Petržalka                27.0    22.0   
4 izbový byt        Bratislava I, Staré Mesto               4.0     8.0   
                    Bratislava II, Podunajské Biskupice     NaN     NaN   
                    Bratislava II, Ružinov                  3.0     6.0   
                    Bratislava II, Vrakuňa                  5.0     8.0   
                    Bratislava III, Nové Mesto              6.0     5.0   
                    Bratislava III, Rača                    3.0     4.0   
                    Bratislava IV, Devínska Nová Ves        NaN     1.0   
                    Bratislava IV, Dúbravka                 2.0     6.0   
                    Bratislava IV, Karlova Ves              NaN     2.0   
                    Bratislava V, Jarovce                   NaN     1.0   
                    Bratislava V, Petržalka                 5.0     2.0   
5 izbový byt a viac Bratislava I, Staré Mesto               1.0     1.0   
                    Bratislava II, Podunajské Biskupice     NaN     NaN   
                    Bratislava II, Ružinov                  NaN     NaN   
                    Bratislava II, Vrakuňa                  1.0     NaN   
                    Bratislava III, Nové Mesto              NaN     1.0   
                    Brati

# AVG PRICE

In [255]:
q12 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':'global',
			'price_avg':{'$avg':'$properties.Cena'}
		}
  
  }
	
]))[0]['price_avg']

print(f"Average price of whole set: {q12}")

Average price of whole set: 195696.64337349398


In [265]:
q12 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                       {'$toString':{'$week':'$scraped timestamp'}}]
            },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek'},
			'price_avg':{'$avg':'$properties.Cena'}
		}
  
  }
	
]))
	
df12 = pd.json_normalize(q12)\
    .rename(columns = {'_id.yearweek':'yearweek'})
df12 = df12.pivot_table(columns = 'yearweek')
df12


yearweek,202019,202020,202021,202022,202023,202024,202025
price_avg,173018.764901,204192.949495,210117.138639,212849.93657,202366.743024,199489.764096,205371.419103


## avg price per category

In [179]:
q13 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'}}  
  }
	
]))
	
df13 = pd.json_normalize(q13)\
    .rename(columns = {'_id.category':'category'})\
    .sort_values('price_avg', ascending = False)
df13 = df13[['category', 'price_avg']]
df13

,category,price_avg
4,5 izbový byt a viac,419887.358025
8,Mezonet,352983.100000
6,4 izbový byt,284583.933610
0,3 izbový byt,216747.491817
9,Apartmán,180511.666667
3,Iný byt,170607.666667
1,2 izbový byt,167853.591915
5,1 izbový byt,120589.865789
7,Dvojgarsónka,114141.851852
2,Garsónka,92551.366667


In [264]:
q13 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
                   {'$toString':{'$week':'$scraped timestamp'}}]
        },
    'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'}}  
  }
	
]))
	
df13 = pd.json_normalize(q13)\
    .rename(columns = {'_id.category':'category', '_id.yearweek':'yearweek'})\
    .sort_values('price_avg', ascending = False)
df13 = df13[['category', 'yearweek', 'price_avg']]
df13 = df13.pivot_table(columns = 'yearweek', index = 'category', values = 'price_avg')
df13

yearweek,202019,202020,202021,202022,202023,202024,202025
category,,,,,,,
1 izbový byt,108238.272727,121568.529412,125771.310811,123858.073826,124075.433962,121607.240741,121510.628378
2 izbový byt,155638.018182,171640.891089,173526.228137,174637.580645,172472.783985,172716.051923,173974.883142
3 izbový byt,199403.040816,231500.184783,230667.123967,232237.522358,222765.972277,220162.782946,225642.856874
4 izbový byt,207915.162791,290136.755556,286667.257812,308314.376682,291154.665025,296450.173267,299656.848958
5 izbový byt a viac,286725.000000,351297.142857,437670.791667,462017.173077,458411.323529,439396.444444,430984.230769
Apartmán,199000.000000,209352.500000,106000.000000,208250.000000,144333.333333,132150.000000,179316.666667
Dvojgarsónka,80163.333333,125000.000000,86888.181818,93355.714286,109037.826087,109226.800000,118162.727273
Garsónka,81050.000000,86499.166667,89461.379310,95577.267606,94394.432836,89240.220779,99508.385714
Iný byt,NaN,258000.000000,126047.666667,144248.000000,163936.500000,168584.333333,161686.500000


## avg price per location

In [180]:
q14 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita'},
			'price_avg':{'$avg':'$properties.Cena'},
              'ads_count':{'$sum':1}}  
  }
	
]))
	
df14 = pd.json_normalize(q14)\
    .rename(columns = {'_id.location':'location'})\
    .sort_values('price_avg', ascending = False)
df14 = df14[['location', 'price_avg', 'ads_count']]
df14.head(30)

,location,price_avg,ads_count
7,"Bratislava IV, Karlova Ves, časť Rovnice",490000.000000,1
59,"Bratislava III, Nové Mesto, časť Koliba",393900.000000,11
41,"Bratislava III, Nové Mesto, časť Vinohrady",290458.125000,8
30,"Bratislava V, Jarovce",288494.736842,38
56,"Bratislava I, Staré Mesto",288286.631741,684
34,"Bratislava IV, Devín",280000.000000,1
36,"Bratislava IV, Karlova Ves, časť Dlhé diely",228893.777778,19
38,"Bratislava III, Nové Mesto, časť Pasienky / Ku...",220354.454545,11
5,"Bratislava III, Rača, časť Rača",219312.777778,9
27,"Bratislava III, Nové Mesto",218688.948864,370


### in top few locations

In [181]:
df15 = df14.loc[df14['location'].isin(locs),:]
df15

,location,price_avg,ads_count
30,"Bratislava V, Jarovce",288494.736842,38
56,"Bratislava I, Staré Mesto",288286.631741,684
27,"Bratislava III, Nové Mesto",218688.948864,370
52,"Bratislava IV, Karlova Ves",190118.116592,229
32,"Bratislava IV, Dúbravka",180214.342342,224
0,"Bratislava II, Ružinov",178311.284195,675
8,"Bratislava III, Rača",164403.483444,156
50,"Bratislava V, Petržalka",160852.384503,698
29,"Bratislava II, Vrakuňa",146824.048276,145
18,"Bratislava II, Podunajské Biskupice",146403.225564,138


In [263]:
q14 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
               {'$toString':{'$week':'$scraped timestamp'}}]
    },
    'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek','location':'$properties.Lokalita'},
			'price_avg':{'$avg':'$properties.Cena'}}  
  }
	
]))
	
df14 = pd.json_normalize(q14)\
    .rename(columns = {'_id.location':'location', '_id.yearweek':'yearweek'})
df14 = df14.loc[df14['location'].isin(locs), ['location', 'yearweek', 'price_avg']]
df14 = df14.pivot_table(columns = 'yearweek', index = 'location', values = 'price_avg')
df14

yearweek,202019,202020,202021,202022,202023,202024,202025
location,,,,,,,
"Bratislava I, Staré Mesto",266180.000000,300163.473684,308008.718310,301933.986799,288856.163701,292053.916129,295315.370492
"Bratislava II, Podunajské Biskupice",128296.666667,159360.000000,156951.160000,150216.065217,153732.549020,152554.107143,147444.193548
"Bratislava II, Ružinov",186169.315789,177472.098039,186262.231293,183537.932624,184409.730897,182327.335548,189403.109890
"Bratislava II, Vrakuňa",152311.111111,159487.647059,134672.605263,152614.149254,145217.632353,143244.544118,152289.962963
"Bratislava III, Nové Mesto",216596.000000,208921.612903,241868.068182,237142.524691,219603.843137,220497.468254,221739.450980
"Bratislava III, Rača",142989.000000,176526.923077,154951.739130,160869.965517,162978.703125,159508.029851,162972.185185
"Bratislava IV, Devínska Nová Ves",133044.444444,125787.500000,135643.333333,142513.333333,137512.564103,144147.058824,142305.142857
"Bratislava IV, Dúbravka",153247.625000,209463.521739,216887.312500,226189.388889,208317.849462,150827.789474,216270.588235
"Bratislava IV, Karlova Ves",226175.000000,216070.000000,223713.720930,213561.341463,214169.146341,184203.645833,203843.191489


## avg price per location and category

In [182]:
q16 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'},
              'ads_count':{'$sum':1}}  
  }
	
]))
	
df16 = pd.json_normalize(q16)\
    .rename(columns = {'_id.location':'location', '_id.category':'category'})\
    .sort_values('price_avg', ascending = False)
df16 = df16[['location', 'category', 'price_avg', 'ads_count']]
df16.head(30)

,location,category,price_avg,ads_count
101,"Bratislava IV, Karlova Ves, časť Dlhé diely",5 izbový byt a viac,636000.000000,2
119,"Bratislava III, Nové Mesto, časť Koliba",4 izbový byt,604775.000000,4
159,"Bratislava I, Staré Mesto",Mezonet,561724.500000,4
113,"Bratislava IV, Dúbravka",5 izbový byt a viac,542192.333333,3
104,"Bratislava III, Nové Mesto",5 izbový byt a viac,495221.111111,10
204,"Bratislava IV, Karlova Ves, časť Rovnice",4 izbový byt,490000.000000,1
28,"Bratislava I, Staré Mesto",5 izbový byt a viac,482088.520000,33
10,"Bratislava IV, Karlova Ves",5 izbový byt a viac,458600.000000,11
98,"Bratislava I, Staré Mesto",4 izbový byt,425956.047059,90
40,"Bratislava V, Jarovce",5 izbový byt a viac,412766.666667,3


In [183]:
df17 = df16.loc[df16['location'].isin(locs),:].pivot(index='location', columns='category', values='price_avg')
df17

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet
location,,,,,,,,,,
"Bratislava I, Staré Mesto",164968.020408,230512.845494,313397.795652,425956.047059,482088.520000,221820.0,NaN,128810.473684,253500.0,561724.5
"Bratislava II, Podunajské Biskupice",92726.153846,119465.434783,173558.057692,171011.250000,258533.333333,NaN,NaN,77993.333333,NaN,NaN
"Bratislava II, Ružinov",122740.150000,164622.070423,214815.611111,285704.750000,270000.000000,130000.0,66233.333333,89084.770833,118900.0,274250.0
"Bratislava II, Vrakuňa",93782.857143,98616.878788,159081.368421,185740.960784,243399.800000,NaN,50327.142857,72870.000000,NaN,NaN
"Bratislava III, Nové Mesto",121010.678571,170390.794872,259294.572816,339289.047619,495221.111111,118000.0,NaN,79475.555556,144248.0,355000.0
"Bratislava III, Rača",110239.913043,153181.486486,208423.448276,207618.750000,295749.500000,NaN,NaN,80960.000000,NaN,NaN
"Bratislava IV, Devínska Nová Ves",104024.736842,132071.666667,147746.363636,239672.727273,NaN,NaN,NaN,73600.000000,NaN,NaN
"Bratislava IV, Dúbravka",111208.950000,139272.710145,187800.362069,279565.744681,542192.333333,NaN,116900.000000,92569.565217,NaN,269000.0
"Bratislava IV, Karlova Ves",117355.416667,166488.260870,189284.252874,251596.562500,458600.000000,NaN,NaN,NaN,NaN,NaN


In [267]:
q16 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
           {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'location':'$properties.Lokalita', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'}}  
  }
	
]))
	
df16 = pd.json_normalize(q16)\
    .rename(columns = {'_id.location':'location', '_id.category':'category', '_id.yearweek':'yearweek'})\
    .sort_values('price_avg', ascending = False)
df16 = df16.loc[df16['location'].isin(locs), ['location', 'yearweek', 'category', 'price_avg']]
df16 = df16.pivot_table(columns = 'yearweek', index = ['category', 'location'], values = 'price_avg')
df16

yearweek                                                        202019  \
category            location                                             
1 izbový byt        Bratislava I, Staré Mesto            150763.333333   
                    Bratislava II, Podunajské Biskupice            NaN   
                    Bratislava II, Ružinov               117925.000000   
                    Bratislava II, Vrakuňa                         NaN   
                    Bratislava III, Nové Mesto           131996.500000   
                    Bratislava III, Rača                 118663.333333   
                    Bratislava IV, Devínska Nová Ves     101166.666667   
                    Bratislava IV, Dúbravka              106000.000000   
                    Bratislava IV, Karlova Ves                     NaN   
                    Bratislava IV, Lamač                           NaN   
                    Bratislava V, Petržalka              105020.000000   
2 izbový byt        Bratislava I, Staré Mesto            209869.500000   
                    Bratislava II, Podunajské Biskupice   91470.000000   
                    Bratislava II, Ružinov               168313.294118   
                    Bratislava II, Vrakuňa                97450.000000   
                    Bratislava III, Nové Mesto           181484.583333   
                    Bratislava III, Rača                 139300.000000   
                    Bratislava IV, Devínska Nová Ves     138490.000000   
                    Bratislava IV, Dúbravka              135307.750000   
                    Bratislava IV, Karlova Ves           187000.000000   
                    Bratislava IV, Lamač                 160000.000000   
                    Bratislava V, Jarovce                          NaN   
                    Bratislava V, Petržalka              158621.888889   
3 izbový byt        Bratislava I, Staré Mesto            350674.750000   
                    Bratislava II, Podunajské Biskupice  201950.000000   
                    Bratislava II, Ružinov               221423.692308   
                    Bratislava II, Vrakuňa                         NaN   
                    Bratislava III, Nové Mesto           277298.000000   
                    Bratislava III, Rača                 144000.000000   
                    Bratislava IV, Devínska Nová Ves     154230.000000   
                    Bratislava IV, Dúbravka              182600.000000   
                    Bratislava IV, Karlova Ves           189080.000000   
                    Bratislava IV, Lamač                 259000.000000   
                    Bratislava V, Jarovce                          NaN   
                    Bratislava V, Petržalka              175013.703704   
4 izbový byt        Bratislava I, Staré Mesto            410670.500000   
                    Bratislava II, Podunajské Biskupice            NaN   
                    Bratislava II, Ružinov               257800.000000   
                    Bratislava II, Vrakuňa               162980.000000   
                    Bratislava III, Nové Mesto           276500.000000   
                    Bratislava III, Rača                 170666.666667   
                    Bratislava IV, Devínska Nová Ves               NaN   
                    Bratislava IV, Dúbravka              175250.000000   
                    Bratislava IV, Karlova Ves                     NaN   
                    Bratislava V, Jarovce                          NaN   
                    Bratislava V, Petržalka              184796.000000   
5 izbový byt a viac Bratislava I, Staré Mesto                      NaN   
                    Bratislava II, Podunajské Biskupice            NaN   
                    Bratislava II, Ružinov                         NaN   
                    Bratislava II, Vrakuňa               299000.000000   
                    Bratislava III, Nové Mesto                     NaN   
                    Bratislava III, Rača                           NaN   
    

## avg price per state

In [184]:
q18 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'state':'$properties.Stav'},
			'price_avg':{'$avg':'$properties.Cena'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df18 = pd.json_normalize(q18)\
    .rename(columns = {'_id.state':'state'})\
    .sort_values('price_avg', ascending = False)
df18 = df18[['state', 'price_avg', 'ads_count']]
df18

,state,price_avg,ads_count
0,None,227647.333333,1131
2,vo výstavbe,222468.416000,128
1,kompletná rekonštrukcia,194577.773547,1543
3,pôvodný,171277.681818,315
4,čiastočná rekonštrukcia,155872.432133,732


In [274]:
q18 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'state':'$properties.Stav'},
			'price_avg':{'$avg':'$properties.Cena'}}
  }
	
]))
	
df18 = pd.json_normalize(q18)\
    .rename(columns = {'_id.state':'state', '_id.yearweek':'yearweek'})\
    .sort_values('price_avg', ascending = False)
df18 = df18[['state', 'yearweek', 'price_avg']]
df18.pivot_table(columns = 'yearweek', index = 'state', values = 'price_avg')

yearweek,202019,202020,202021,202022,202023,202024,202025
state,,,,,,,
kompletná rekonštrukcia,172464.072165,201802.266055,205444.971875,208269.348432,198455.237864,197578.19637,204344.004702
pôvodný,157840.000000,175404.760000,166668.276923,166808.090164,162612.756098,160219.32500,180154.263636
vo výstavbe,223046.692308,277541.500000,223873.904762,226828.833333,224019.197531,201620.26087,229243.562500
čiastočná rekonštrukcia,143954.814815,148919.200000,153575.811475,160213.557692,159855.927273,161700.80000,160365.840278


## avg price per state and category

In [185]:
q19 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df19 = pd.json_normalize(q19)\
    .rename(columns = {'_id.state':'state', '_id.category':'category'})\
    .sort_values(['state', 'price_avg'], ascending = False)
df19 = df19[['state', 'category', 'price_avg', 'ads_count']]
df19 = df19.pivot(index = 'state', columns = 'category', values = 'price_avg')
df19

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet
state,,,,,,,,,,
NaN,118899.281250,178019.258427,259843.213235,369638.673333,447779.302326,193730.0,132833.333333,110480.937500,253500.0,561724.500000
kompletná rekonštrukcia,130637.851240,170162.157424,220584.628352,268249.760000,471983.333333,112900.0,125620.588235,96450.261538,114075.0,286833.333333
pôvodný,110686.060606,145272.638889,198219.912000,240098.093750,385999.875000,106000.0,47745.000000,82603.971429,NaN,NaN
vo výstavbe,129015.285714,180593.403846,234975.375000,330789.937500,482315.400000,243325.0,NaN,170850.000000,144248.0,342691.000000
čiastočná rekonštrukcia,111178.654762,141346.207650,172495.217712,200038.247706,276383.846154,NaN,93238.000000,80693.617021,NaN,NaN


In [276]:
q19 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'}}
		
  
  }
	
]))
	
df19 = pd.json_normalize(q19)\
    .rename(columns = {'_id.state':'state', '_id.category':'category', '_id.yearweek':'yearweek'})
df19 = df19[['state', 'category', 'yearweek', 'price_avg']]
df19 = df19.pivot_table(index = ['state', 'category'], columns = 'yearweek', values = 'price_avg')
df19

yearweek                                            202019         202020  \
state                   category                                            
kompletná rekonštrukcia 1 izbový byt         115370.000000  127459.333333   
                        2 izbový byt         157952.377778  167323.078947   
                        3 izbový byt         193389.600000  233022.285714   
                        4 izbový byt         225490.000000  301100.000000   
                        5 izbový byt a viac            NaN            NaN   
                        Apartmán                       NaN            NaN   
                        Dvojgarsónka         124500.000000  125000.000000   
                        Garsónka              90750.000000   98900.000000   
                        Iný byt                        NaN            NaN   
                        Mezonet                        NaN  129000.000000   
pôvodný                 1 izbový byt          99225.000000  109548.000000   
                        2 izbový byt         177963.333333  152718.000000   
                        3 izbový byt         182271.428571  211755.444444   
                        4 izbový byt         209380.000000  172000.000000   
                        5 izbový byt a viac            NaN  495000.000000   
                        Apartmán                       NaN            NaN   
                        Dvojgarsónka          53990.000000            NaN   
                        Garsónka              82450.000000   78495.000000   
vo výstavbe             1 izbový byt         149993.000000  119000.000000   
                        2 izbový byt         164497.333333  181577.500000   
                        3 izbový byt         210506.666667  233066.000000   
                        4 izbový byt         212000.000000  291950.000000   
                        5 izbový byt a viac  490000.000000  797531.000000   
                        Apartmán                       NaN  243325.000000   
                        Garsónka                       NaN            NaN   
                        Iný byt                        NaN            NaN   
                        Mezonet              291082.000000  394300.000000   
čiastočná rekonštrukcia 1 izbový byt         112212.857143  103600.000000   
                        2 izbový byt         134566.000000  130616.250000   
                        3 izbový byt         169871.666667  186718.750000   
                        4 izbový byt         158989.000000  169162.500000   
                        5 izbový byt a viac            NaN  170000.000000   
                        Dvojgarsónka          62000.000000            NaN   
                        Garsónka              86666.666667   81825.000000   

yearweek                                            202021         202022  \
state                   category                                            
kompletná rekonštrukcia 1 izbový byt         132089.863636  133210.641026   
                        2 izbový byt         181926.936937  177634.266990   
                        3 izbový byt         226441.813559  232170.312796   
                        4 izbový byt         266241.632653  287298.718310   
                        5 izbový byt a viac  404300.000000  530914.285714   
                        Apartmán                       NaN            NaN   
                        Dvojgarsónka         129000.000000  116144.444444   
                        Garsónka             100984.615385  103358.172414   
                        Iný byt              114075.000000            NaN   
                        Mezonet              269000.000000  306750.000000   
pôvodný                 1 izbový byt         117954.545455  112211.764706   
                        2 izbový byt         154213.076923  152860.344828   
                        3 izbový byt         176044.736842  199542.051282   
                        4 izbový byt         219444.916667  209367.07142

## avg price per state and location

In [186]:
q20 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita', 'state':'$properties.Stav'},
			'price_avg':{'$avg':'$properties.Cena'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df20 = pd.json_normalize(q20)\
    .rename(columns = {'_id.location':'location', '_id.state':'state'})\
    .sort_values('price_avg', ascending = False)
df20 = df20[['location', 'state', 'price_avg', 'ads_count']]
df20 = df20.loc[df20['location'].isin(locs),:].pivot(index = 'location', columns = 'state', values = 'price_avg')
df20

state,NaN,kompletná rekonštrukcia,pôvodný,vo výstavbe,čiastočná rekonštrukcia
location,,,,,
"Bratislava I, Staré Mesto",325849.633987,283988.783934,257136.455882,291267.875000,252672.152542
"Bratislava II, Podunajské Biskupice",156656.842105,147251.459016,109481.250000,159000.000000,139270.000000
"Bratislava II, Ružinov",200282.668367,175083.149466,146395.975000,190030.960000,148545.824176
"Bratislava II, Vrakuňa",183878.659091,142576.384615,104093.052632,237700.000000,120763.564103
"Bratislava III, Nové Mesto",285816.051724,197742.986301,163475.862069,175522.583333,162162.972973
"Bratislava III, Rača",203174.404255,143015.957447,155857.307692,NaN,145215.774194
"Bratislava IV, Devínska Nová Ves",133230.769231,151666.818182,171244.444444,155666.666667,123548.571429
"Bratislava IV, Dúbravka",220528.728571,146235.194805,133886.470588,389280.363636,143666.148936
"Bratislava IV, Karlova Ves",255859.811321,173554.883721,188000.000000,317000.000000,156042.957746


In [278]:
q20 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
        {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'location':'$properties.Lokalita', 'state':'$properties.Stav'},
			'price_avg':{'$avg':'$properties.Cena'}}
		
  
  }
	
]))
	
df20 = pd.json_normalize(q20)\
    .rename(columns = {'_id.location':'location', '_id.state':'state', '_id.yearweek':'yearweek'})
df20 = df20[['location', 'yearweek', 'state', 'price_avg']]
df20 = df20.loc[df20['location'].isin(locs),:].pivot_table(index = ['state', 'location'], columns = 'yearweek', values = 'price_avg')
df20

yearweek                                                            202019  \
state                   location                                             
kompletná rekonštrukcia Bratislava I, Staré Mesto            261579.200000   
                        Bratislava II, Podunajské Biskupice   86963.333333   
                        Bratislava II, Ružinov               169522.941176   
                        Bratislava II, Vrakuňa               112450.000000   
                        Bratislava III, Nové Mesto           174250.000000   
                        Bratislava III, Rača                 135296.666667   
                        Bratislava IV, Devínska Nová Ves     146333.333333   
                        Bratislava IV, Dúbravka              157721.111111   
                        Bratislava IV, Karlova Ves           155750.000000   
                        Bratislava IV, Lamač                           NaN   
                        Bratislava V, Jarovce                          NaN   
                        Bratislava V, Petržalka              168533.850000   
pôvodný                 Bratislava I, Staré Mesto            249950.000000   
                        Bratislava II, Podunajské Biskupice  104990.000000   
                        Bratislava II, Ružinov               385000.000000   
                        Bratislava II, Vrakuňa               149900.000000   
                        Bratislava III, Nové Mesto           154950.000000   
                        Bratislava III, Rača                 164000.000000   
                        Bratislava IV, Devínska Nová Ves      95000.000000   
                        Bratislava IV, Dúbravka                        NaN   
                        Bratislava IV, Karlova Ves           239000.000000   
                        Bratislava IV, Lamač                           NaN   
                        Bratislava V, Petržalka              130380.000000   
vo výstavbe             Bratislava I, Staré Mesto                      NaN   
                        Bratislava II, Podunajské Biskupice            NaN   
                        Bratislava II, Ružinov               182922.000000   
                        Bratislava II, Vrakuňa               212000.000000   
                        Bratislava III, Nové Mesto           187084.333333   
                        Bratislava IV, Devínska Nová Ves     162500.000000   
                        Bratislava IV, Dúbravka                        NaN   
                        Bratislava IV, Karlova Ves           490000.000000   
                        Bratislava V, Jarovce                          NaN   
                        Bratislava V, Petržalka              246281.000000   
čiastočná rekonštrukcia Bratislava I, Staré Mesto            197747.500000   
                        Bratislava II, Podunajské Biskupice  164900.000000   
                        Bratislava II, Ružinov               159000.000000   
                        Bratislava II, Vrakuňa               116666.666667   
                        Bratislava III, Nové Mesto           163538.000000   
                        Bratislava III, Rača                 110000.000000   
                        Bratislava IV, Devínska Nová Ves     109950.000000   
                        Bratislava IV, Dúbravka              129333.333333   
                        Bratislava IV, Karlova Ves           182966.666667   
                        Bratislava IV, Lamač                           NaN   
                        Bratislava V, Petržalka              138286.000000   

yearweek                                                            202020  \
state                   location                                             
kompletná rekonštrukcia Bratislava I, Staré Mesto            303882.142857   
                        Bratislava II, Podunajské Biskupice  199000.000000   
                        Bratislava II, Ružinov               171575.200000   
                     

## avg price per state, location and category

In [187]:
q21 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita', 'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df21 = pd.json_normalize(q21)\
    .rename(columns = {'_id.location':'location', '_id.state':'state', '_id.category':'category'})\
    .sort_values('price_avg', ascending = False)
df21 = df21.loc[df21['location'].isin(locs),:].pivot_table(index = ['location', 'category'], columns = 'state', values = ['price_avg', 'ads_count'])
df21.columns = df21.columns.swaplevel(0, 1)
df21.sort_index(axis=1, level=0, inplace=True)
df21

state                                                   kompletná rekonštrukcia  \
                                                                      ads_count   
location                            category                                      
Bratislava I, Staré Mesto           1 izbový byt                           31.0   
                                    2 izbový byt                          132.0   
                                    3 izbový byt                          137.0   
                                    4 izbový byt                           44.0   
                                    5 izbový byt a viac                    15.0   
                                    Apartmán                                1.0   
                                    Garsónka                               14.0   
                                    Mezonet                                 NaN   
Bratislava II, Podunajské Biskupice 1 izbový byt                            1.0   
                                    2 izbový byt                           29.0   
                                    3 izbový byt                           23.0   
                                    4 izbový byt                            8.0   
                                    5 izbový byt a viac                     NaN   
                                    Garsónka                                2.0   
Bratislava II, Ružinov              1 izbový byt                           28.0   
                                    2 izbový byt                          135.0   
                                    3 izbový byt                           82.0   
                                    4 izbový byt                           25.0   
                                    5 izbový byt a viac                     NaN   
                                    Dvojgarsónka                            1.0   
                                    Garsónka                               13.0   
                                    Iný byt                                 1.0   
                                    Mezonet                                 4.0   
Bratislava II, Vrakuňa              1 izbový byt                            1.0   
                                    2 izbový byt                           10.0   
                                    3 izbový byt                           11.0   
                                    4 izbový byt                           16.0   
                                    5 izbový byt a viac                     NaN   
                                    Dvojgarsónka                            1.0   
                                    Garsónka                                NaN   
Bratislava III, Nové Mesto          1 izbový byt                            7.0   
                                    2 izbový byt                           73.0   
                                    3 izbový byt                           54.0   
                                    4 izbový byt                           18.0   
                                    5 izbový byt a viac                     NaN   
                                    Apartmán                                NaN   
                                    Garsónka                                1.0   
                                    Iný byt                                 NaN   
                                    Mezonet                                 1.0   
Bratislava III, Rača                1 izbový byt                           13.0   
                                    2 izbový byt                           30.0   
                                    3 izbový byt                            5.0   
                                    4 izbový byt                            2.0   
                                    5 izbový byt a viac                     NaN   
                                    Garsónka                                1.0   
Bratislava IV, D

In [279]:
q21 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'location':'$properties.Lokalita', 'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_avg':{'$avg':'$properties.Cena'}}
		
  
  }
	
]))
	
df21 = pd.json_normalize(q21)\
    .rename(columns = {'_id.location':'location', '_id.state':'state', '_id.category':'category', '_id.yearweek':'yearweek'})
df21 = df21.loc[df21['location'].isin(locs),:].pivot_table(index = ['category', 'state', 'location'], columns = 'yearweek', values = 'price_avg')
df21

yearweek                                                                                202019  \
category            state                   location                                             
1 izbový byt        kompletná rekonštrukcia Bratislava I, Staré Mesto            179300.000000   
                                            Bratislava II, Podunajské Biskupice            NaN   
                                            Bratislava II, Ružinov               113900.000000   
                                            Bratislava II, Vrakuňa                         NaN   
                                            Bratislava III, Nové Mesto                     NaN   
                                            Bratislava III, Rača                 115990.000000   
                                            Bratislava IV, Devínska Nová Ves               NaN   
                                            Bratislava IV, Dúbravka                        NaN   
                                            Bratislava IV, Karlova Ves                     NaN   
                                            Bratislava IV, Lamač                           NaN   
                                            Bratislava V, Petržalka              106900.000000   
                    pôvodný                 Bratislava I, Staré Mesto                      NaN   
                                            Bratislava II, Podunajské Biskupice            NaN   
                                            Bratislava II, Ružinov                         NaN   
                                            Bratislava III, Nové Mesto                     NaN   
                                            Bratislava IV, Devínska Nová Ves      95000.000000   
                                            Bratislava IV, Dúbravka                        NaN   
                                            Bratislava IV, Karlova Ves                     NaN   
                                            Bratislava IV, Lamač                           NaN   
                                            Bratislava V, Petržalka              113000.000000   
                    vo výstavbe             Bratislava II, Ružinov                         NaN   
                                            Bratislava III, Nové Mesto           149993.000000   
                    čiastočná rekonštrukcia Bratislava I, Staré Mesto            136495.000000   
                                            Bratislava II, Podunajské Biskupice            NaN   
                                            Bratislava II, Ružinov                         NaN   
                                            Bratislava II, Vrakuňa                         NaN   
                                            Bratislava III, Nové Mesto                     NaN   
                                            Bratislava III, Rača                 110000.000000   
                                            Bratislava IV, Devínska Nová Ves      90000.000000   
                                            Bratislava IV, Dúbravka                        NaN   
                                            Bratislava IV, Karlova Ves                     NaN   
                                            Bratislava IV, Lamač                           NaN   
                                            Bratislava V, Petržalka               97700.000000   
2 izbový byt        kompletná rekonštrukcia Bratislava I, Staré Mesto            228936.250000   
                                            Bratislava II, Podunajské Biskupice   86963.333333   
                                            Bratislava II, Ružinov               171419.000000   
                                            Bratislava II, Vrakuňa                59900.000000   
                                            Bratislava III, Nové Mesto           148200.000000   
                                            Bratislava III, Rača                

# AVG M2 PRICE

In [188]:
q12 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':'global',
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}
		}
  
  }
	
]))[0]['price_m2_avg']
	
print(f"Average price of whole set: {q12}")


Average price of whole set: 2920.412510110542


In [285]:
q12 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':'$yearweek',
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}
		}
  
  }
	
]))
	
df12 = pd.json_normalize(q12)\
    .rename(columns = {'_id':'yearweek'})
df12 = df12.pivot_table(columns = 'yearweek')
df12

yearweek,202019,202020,202021,202022,202023,202024,202025
price_m2_avg,2651.734219,3034.081633,2952.55799,2984.977139,3000.809275,2954.799349,3002.668844


## avg m2 price per category

In [189]:
q13 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}
		
  
  }
	
]))
	
df13 = pd.json_normalize(q13)\
    .rename(columns = {'_id.category':'category'})\
    .sort_values('price_m2_avg', ascending = False)
df13 = df13[['category', 'price_m2_avg']]
df13

,category,price_m2_avg
2,Garsónka,3466.414773
4,1 izbový byt,3204.697613
8,Apartmán,3179.888889
9,2 izbový byt,3055.507291
5,Mezonet,2963.000000
0,Iný byt,2786.666667
1,3 izbový byt,2744.680328
7,Dvojgarsónka,2717.944444
6,4 izbový byt,2646.722105
3,5 izbový byt a viac,2592.189873


In [288]:
q13 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}
		
  
  }
	
]))
	
df13 = pd.json_normalize(q13)\
    .rename(columns = {'_id.category':'category', '_id.yearweek':'yearweek'})
df13 = df13[['category', 'yearweek', 'price_m2_avg']]
df13 = df13.pivot_table(columns = 'yearweek', index = 'category', values = 'price_m2_avg')
df13

yearweek,202019,202020,202021,202022,202023,202024,202025
category,,,,,,,
1 izbový byt,2976.757576,3278.545455,3248.878378,3300.684564,3337.509434,3224.720497,3275.850340
2 izbový byt,2820.899083,3091.257426,3121.730038,3128.319312,3155.889513,3108.622093,3146.059615
3 izbový byt,2558.255102,2828.813187,2851.414938,2825.065173,2797.269841,2756.042636,2822.516008
4 izbový byt,2078.837209,2971.266667,2628.619048,2707.594470,2707.375000,2711.250000,2786.836842
5 izbový byt a viac,1902.750000,3136.142857,2759.166667,2746.450980,2831.303030,2390.576923,2704.473684
Apartmán,3062.000000,3737.000000,1828.000000,2828.000000,2783.000000,3255.833333,3053.166667
Dvojgarsónka,1891.333333,2604.000000,2129.909091,2279.380952,2660.608696,2631.240000,2740.500000
Garsónka,3557.750000,3740.181818,3390.827586,3593.000000,3581.630769,3578.961039,3564.367647
Iný byt,NaN,2345.000000,2679.000000,3741.500000,2953.500000,2801.666667,2933.250000


## avg m2 price per location

In [190]:
q14 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df14 = pd.json_normalize(q14)\
    .rename(columns = {'_id.location':'location'})\
    .sort_values('price_m2_avg', ascending = False)
df14 = df14[['location', 'price_m2_avg', 'ads_count']]
df14.head(30)

,location,price_m2_avg,ads_count
55,"Bratislava I, Staré Mesto",3731.911765,684
40,"Bratislava III, Nové Mesto, časť Vinohrady",3413.875000,8
37,"Bratislava III, Nové Mesto, časť Pasienky / Ku...",3256.818182,11
38,"Bratislava V, Rusovce",3086.500000,4
27,"Bratislava III, Nové Mesto",3061.455840,370
12,"Bratislava II, Ružinov, časť Nivy",3061.090909,33
64,"Bratislava II, Ružinov",3039.471669,675
50,"Bratislava II, Ružinov, časť Trnávka",3023.100000,10
13,"Bratislava III, Nové Mesto, časť Kramáre",2905.181818,11
15,"Bratislava II, Ružinov, časť Štrkovec",2895.750000,13


### in top few locations

In [191]:
df15 = df14.loc[df14['location'].isin(locs),:]
df15

,location,price_m2_avg,ads_count
55,"Bratislava I, Staré Mesto",3731.911765,684
27,"Bratislava III, Nové Mesto",3061.455840,370
64,"Bratislava II, Ružinov",3039.471669,675
31,"Bratislava IV, Dúbravka",2816.739726,224
7,"Bratislava III, Rača",2789.412162,156
51,"Bratislava IV, Karlova Ves",2759.931818,229
4,"Bratislava IV, Lamač",2722.742857,35
48,"Bratislava V, Petržalka",2698.998527,698
25,"Bratislava IV, Devínska Nová Ves",2381.287234,99
28,"Bratislava V, Jarovce",2321.921053,38


In [292]:
q14 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'location':'$properties.Lokalita'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}
		
  
  }
	
]))
	
df14 = pd.json_normalize(q14)\
    .rename(columns = {'_id.location':'location', '_id.yearweek':'yearweek'})
df14 = df14.loc[df14['location'].isin(locs), ['location', 'yearweek','price_m2_avg']]
df14 = df14.pivot_table(columns = 'yearweek', index = 'location', values = 'price_m2_avg')
df14

yearweek,202019,202020,202021,202022,202023,202024,202025
location,,,,,,,
"Bratislava I, Staré Mesto",3591.027027,3935.526316,3764.312057,3769.890000,3758.665468,3736.048701,3792.280528
"Bratislava II, Podunajské Biskupice",2348.666667,2011.200000,2106.520000,2158.826087,2279.137255,2282.071429,2316.306452
"Bratislava II, Ružinov",3047.894737,2988.920000,3023.204082,3115.953405,3143.622074,3113.759197,3082.871324
"Bratislava II, Vrakuňa",1854.777778,2011.941176,1896.105263,2047.833333,2041.867647,2013.382353,2088.415094
"Bratislava III, Nové Mesto",3081.370370,3212.258065,3170.818182,3085.598765,3095.986928,3035.198413,3144.144737
"Bratislava III, Rača",2875.400000,2797.769231,2690.217391,2797.982456,2824.888889,2830.666667,2801.730769
"Bratislava IV, Devínska Nová Ves",2390.555556,2238.000000,2213.222222,2334.800000,2384.871795,2364.205882,2481.771429
"Bratislava IV, Dúbravka",2787.000000,2817.173913,3006.437500,3007.328571,2968.296703,2651.080000,2955.083333
"Bratislava IV, Karlova Ves",2711.500000,2975.400000,2747.904762,2729.575000,2784.500000,2729.252632,2698.170213


## avg m2 price per location and category

In [192]:
q16 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df16 = pd.json_normalize(q16)\
    .rename(columns = {'_id.location':'location', '_id.category':'category'})\
    .sort_values('price_m2_avg', ascending = False)
df16 = df16[['location', 'category', 'price_m2_avg', 'ads_count']]
df16.head(30)

,location,category,price_m2_avg,ads_count
17,"Bratislava III, Nové Mesto, časť Pasienky / Ku...",Garsónka,5550.000000,1
35,"Bratislava I, Staré Mesto",Garsónka,4633.555556,20
38,"Bratislava IV, Dúbravka",5 izbový byt a viac,4315.333333,3
37,"Bratislava III, Nové Mesto",Garsónka,4199.111111,9
106,"Bratislava II, Ružinov, časť Nivy",Garsónka,4164.000000,2
174,"Bratislava I, Staré Mesto",1 izbový byt,4090.142857,49
16,"Bratislava I, Staré Mesto",2 izbový byt,3955.224138,244
1,"Bratislava V, Rusovce",Garsónka,3800.000000,1
227,"Bratislava I, Staré Mesto",Mezonet,3754.500000,4
91,"Bratislava II, Ružinov",Garsónka,3750.604167,48


In [193]:
df17 = df16.loc[df16['location'].isin(locs),:].pivot(index='location', columns='category', values='price_m2_avg')
df17

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet
location,,,,,,,,,,
"Bratislava I, Staré Mesto",4090.142857,3955.224138,3505.517391,3592.416667,2971.440000,3736.5,NaN,4633.555556,2304.5,3754.5
"Bratislava II, Podunajské Biskupice",2853.846154,2392.521739,2153.326923,1855.687500,1351.000000,NaN,NaN,3328.000000,NaN,NaN
"Bratislava II, Ružinov",3417.012821,2996.978723,2847.288889,2837.127273,2288.000000,2826.0,1907.666667,3750.604167,2642.0,2759.5
"Bratislava II, Vrakuňa",2427.000000,2047.666667,2069.052632,2079.857143,1962.400000,NaN,1318.857143,2094.250000,NaN,NaN
"Bratislava III, Nové Mesto",3256.250000,3036.200000,3049.563107,2917.023810,2650.333333,2327.0,NaN,4199.111111,3741.5,2383.0
"Bratislava III, Rača",3095.272727,2958.851351,2430.379310,2235.625000,2352.500000,NaN,NaN,2965.200000,NaN,NaN
"Bratislava IV, Devínska Nová Ves",2493.315789,2549.138889,2087.318182,2172.727273,NaN,NaN,NaN,2479.666667,NaN,NaN
"Bratislava IV, Dúbravka",2873.600000,2851.671642,2598.706897,2763.936170,4315.333333,NaN,3340.000000,3138.909091,NaN,2360.0
"Bratislava IV, Karlova Ves",3023.458333,3088.521739,2596.000000,2367.827586,2546.800000,NaN,NaN,NaN,NaN,NaN


In [295]:
q16 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'location':'$properties.Lokalita', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}	
  
  }
	
]))
	
df16 = pd.json_normalize(q16)\
    .rename(columns = {'_id.location':'location', '_id.category':'category', '_id.yearweek':'yearweek'})
df16 = df16.loc[df16['location'].isin(locs), ['location', 'category', 'yearweek', 'price_m2_avg']]
df16 = df16.pivot_table(columns = 'yearweek', index = ['category', 'location'], values = 'price_m2_avg')
df16

yearweek                                                      202019  \
category            location                                           
1 izbový byt        Bratislava I, Staré Mesto            4069.000000   
                    Bratislava II, Podunajské Biskupice          NaN   
                    Bratislava II, Ružinov               3557.750000   
                    Bratislava II, Vrakuňa                       NaN   
                    Bratislava III, Nové Mesto           3166.000000   
                    Bratislava III, Rača                 3181.000000   
                    Bratislava IV, Devínska Nová Ves     2511.000000   
                    Bratislava IV, Dúbravka              3419.000000   
                    Bratislava IV, Karlova Ves                   NaN   
                    Bratislava IV, Lamač                         NaN   
                    Bratislava V, Petržalka              3073.400000   
2 izbový byt        Bratislava I, Staré Mesto            3599.550000   
                    Bratislava II, Podunajské Biskupice  2501.000000   
                    Bratislava II, Ružinov               2983.411765   
                    Bratislava II, Vrakuňa               1769.000000   
                    Bratislava III, Nové Mesto           2951.250000   
                    Bratislava III, Rača                 3429.333333   
                    Bratislava IV, Devínska Nová Ves     2611.500000   
                    Bratislava IV, Dúbravka              2933.000000   
                    Bratislava IV, Karlova Ves           3307.000000   
                    Bratislava IV, Lamač                 3019.000000   
                    Bratislava V, Jarovce                        NaN   
                    Bratislava V, Petržalka              2882.000000   
3 izbový byt        Bratislava I, Staré Mesto            3903.500000   
                    Bratislava II, Podunajské Biskupice  2044.000000   
                    Bratislava II, Ružinov               2946.153846   
                    Bratislava II, Vrakuňa                       NaN   
                    Bratislava III, Nové Mesto           3152.250000   
                    Bratislava III, Rača                 2400.000000   
                    Bratislava IV, Devínska Nová Ves     2189.750000   
                    Bratislava IV, Dúbravka              2634.600000   
                    Bratislava IV, Karlova Ves           2499.800000   
                    Bratislava IV, Lamač                 2398.000000   
                    Bratislava V, Jarovce                        NaN   
                    Bratislava V, Petržalka              2434.444444   
4 izbový byt        Bratislava I, Staré Mesto            2616.500000   
                    Bratislava II, Podunajské Biskupice          NaN   
                    Bratislava II, Ružinov               2698.666667   
                    Bratislava II, Vrakuňa               1882.800000   
                    Bratislava III, Nové Mesto           2920.500000   
                    Bratislava III, Rača                 2174.333333   
                    Bratislava IV, Devínska Nová Ves             NaN   
                    Bratislava IV, Dúbravka              2341.000000   
                    Bratislava IV, Karlova Ves                   NaN   
                    Bratislava V, Jarovce                        NaN   
                    Bratislava V, Petržalka              2294.400000   
5 izbový byt a viac Bratislava I, Staré Mesto                    NaN   
                    Bratislava II, Podunajské Biskupice          NaN   
                    Bratislava II, Ružinov                       NaN   
                    Bratislava II, Vrakuňa               2151.000000   
                    Bratislava III, Nové Mesto                   NaN   
                    Bratislava III, Rača                         NaN   
                    Bratislava IV, Dúbravka                      NaN   
                    Bratislava IV, Karlo

## avg m2 price per state

In [194]:
q18 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'state':'$properties.Stav'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df18 = pd.json_normalize(q18)\
    .rename(columns = {'_id.state':'state'})\
    .sort_values('price_m2_avg', ascending = False)
df18 = df18[['state', 'price_m2_avg', 'ads_count']]
df18

,state,price_m2_avg,ads_count
2,vo výstavbe,3349.935484,128
1,kompletná rekonštrukcia,3055.993284,1543
0,None,2989.821229,1131
3,pôvodný,2702.283388,315
4,čiastočná rekonštrukcia,2552.972028,732


In [298]:
q18 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'state':'$properties.Stav'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}
		
  
  }
	
]))
	
df18 = pd.json_normalize(q18)\
    .rename(columns = {'_id.state':'state', '_id.yearweek':'yearweek'})
df18 = df18[['state', 'yearweek', 'price_m2_avg']]
df18 = df18.pivot_table(columns = 'yearweek', index = 'state', values = 'price_m2_avg')
df18

yearweek,202019,202020,202021,202022,202023,202024,202025
state,,,,,,,
kompletná rekonštrukcia,2857.520833,3122.879630,3093.451411,3124.700000,3089.507341,3063.528053,3108.707547
pôvodný,2499.181818,3044.320000,2645.876923,2795.347107,2753.829268,2670.633333,2700.172727
vo výstavbe,2999.000000,3134.916667,3530.585366,3351.579545,3497.405063,3350.117647,3507.329114
čiastočná rekonštrukcia,2451.351852,2609.938776,2479.581967,2544.766537,2583.596364,2575.481848,2588.339161


## avg m2 price per state and category

In [195]:
q19 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df19 = pd.json_normalize(q19)\
    .rename(columns = {'_id.state':'state', '_id.category':'category'})\
    .sort_values(['state', 'price_m2_avg'], ascending = False)
df19 = df19[['state', 'category', 'price_m2_avg', 'ads_count']]
df19 = df19.pivot(index = 'state', columns = 'category', values = 'price_m2_avg')
df19

category,1 izbový byt,2 izbový byt,3 izbový byt,4 izbový byt,5 izbový byt a viac,Apartmán,Dvojgarsónka,Garsónka,Iný byt,Mezonet
state,,,,,,,,,,
NaN,3056.338583,3052.196388,2901.941176,2924.945946,2523.560976,3312.666667,3362.333333,3434.100000,2304.5,3754.500000
kompletná rekonštrukcia,3519.250000,3198.366248,2859.124760,2684.430233,2934.000000,4181.000000,3004.264706,3645.390625,2314.0,2630.166667
pôvodný,2949.939394,2603.830986,2552.600000,2498.343750,2411.750000,1828.000000,1251.500000,3563.828571,NaN,NaN
vo výstavbe,3661.500000,3485.846154,3112.093750,3057.333333,3562.400000,2734.000000,NaN,3051.000000,3741.5,3170.000000
čiastočná rekonštrukcia,3001.168675,2675.427778,2410.800000,2192.333333,2231.000000,NaN,2135.600000,3173.391304,NaN,NaN


In [300]:
q19 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}
		
  
  }
	
]))
	
df19 = pd.json_normalize(q19)\
    .rename(columns = {'_id.state':'state', '_id.category':'category', '_id.yearweek': 'yearweek'})
df19 = df19[['state', 'category', 'yearweek', 'price_m2_avg']]
df19 = df19.pivot_table(index = ['category', 'state'], columns = 'yearweek', values = 'price_m2_avg')
df19

yearweek                                          202019       202020  \
category            state                                               
1 izbový byt        kompletná rekonštrukcia  3216.714286  3677.571429   
                    pôvodný                  2471.750000  3097.200000   
                    vo výstavbe              3409.000000  3719.000000   
                    čiastočná rekonštrukcia  3085.571429  2863.400000   
2 izbový byt        kompletná rekonštrukcia  3013.136364  3076.342105   
                    pôvodný                  3074.333333  2688.400000   
                    vo výstavbe              3139.333333  3298.000000   
                    čiastočná rekonštrukcia  2480.000000  2671.125000   
3 izbový byt        kompletná rekonštrukcia  2698.433333  2931.514286   
                    pôvodný                  2255.714286  2719.222222   
                    vo výstavbe              3002.500000  2983.500000   
                    čiastočná rekonštrukcia  2297.222222  2460.800000   
4 izbový byt        kompletná rekonštrukcia  2353.666667  3057.117647   
                    pôvodný                  2250.200000  3416.666667   
                    vo výstavbe              1927.000000  2587.000000   
                    čiastočná rekonštrukcia  1994.900000  2072.750000   
5 izbový byt a viac kompletná rekonštrukcia          NaN          NaN   
                    pôvodný                          NaN  3474.000000   
                    vo výstavbe              2579.000000  4132.000000   
                    čiastočná rekonštrukcia          NaN  4331.000000   
Apartmán            kompletná rekonštrukcia          NaN          NaN   
                    pôvodný                          NaN          NaN   
                    vo výstavbe                      NaN  2734.000000   
Dvojgarsónka        kompletná rekonštrukcia  2767.000000  2604.000000   
                    pôvodný                  1317.000000          NaN   
                    čiastočná rekonštrukcia  1590.000000          NaN   
Garsónka            kompletná rekonštrukcia  3631.500000  4721.500000   
                    pôvodný                  3757.000000  4491.500000   
                    vo výstavbe                      NaN          NaN   
                    čiastočná rekonštrukcia  3561.666667  3251.750000   
Iný byt             kompletná rekonštrukcia          NaN          NaN   
                    vo výstavbe                      NaN          NaN   
Mezonet             kompletná rekonštrukcia          NaN  2263.000000   
                    vo výstavbe              3639.000000  2701.000000   

yearweek                                          202021       202022  \
category            state                                               
1 izbový byt        kompletná rekonštrukcia  3539.545455  3613.102564   
                    pôvodný                  3138.090909  3193.764706   
                    vo výstavbe              3655.200000  3639.111111   
                    čiastočná rekonštrukcia  3129.125000  3126.681818   
2 izbový byt        kompletná rekonštrukcia  3334.558559  3328.521951   
                    pôvodný                  2656.538462  2719.214286   
                    vo výstavbe              3542.142857  3470.567568   
                    čiastočná rekonštrukcia  2684.090909  2670.968254   
3 izbový byt        kompletná rekonštrukcia  2904.350427  2890.604762   
                    pôvodný                  2376.578947  2580.923077   
                    vo výstavbe              3286.818182  3058.478261   
                    čiastočná rekonštrukcia  2312.600000  2421.530000   
4 izbový byt        kompletná rekonštrukcia  2706.836735  2719.275362   
                    pôvodný                  2471.333333  2468.000000   
                    vo výstavbe              4758.000000  3093.181818   
                    čiastočná rekonštrukcia  2125.153846  2249.575000   
5 izbový byt a viac kompletná rekonštrukcia  2859.500000 

## avg m2 price per state and location

In [196]:
q20 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita', 'state':'$properties.Stav'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df20 = pd.json_normalize(q20)\
    .rename(columns = {'_id.location':'location', '_id.state':'state'})\
    .sort_values('price_m2_avg', ascending = False)
df20 = df20[['location', 'state', 'price_m2_avg', 'ads_count']]
df20 = df20.loc[df20['location'].isin(locs),:].pivot(index = 'location', columns = 'state', values = 'price_m2_avg')
df20

state,NaN,kompletná rekonštrukcia,pôvodný,vo výstavbe,čiastočná rekonštrukcia
location,,,,,
"Bratislava I, Staré Mesto",3967.493421,3848.016620,3091.941176,3867.857143,3125.775862
"Bratislava II, Podunajské Biskupice",2518.000000,2343.442623,2153.375000,1880.500000,1801.458333
"Bratislava II, Ružinov",3121.572165,2981.537634,3011.923077,3475.860000,2814.098901
"Bratislava II, Vrakuňa",2279.090909,2144.270270,1634.894737,2297.750000,1862.769231
"Bratislava III, Nové Mesto",3004.258621,3092.260274,3079.448276,3387.750000,2888.805556
"Bratislava III, Rača",2814.155556,3081.680851,2296.769231,NaN,2721.366667
"Bratislava IV, Devínska Nová Ves",2589.720000,2391.954545,2229.222222,2544.333333,2250.828571
"Bratislava IV, Dúbravka",3095.357143,2726.520000,2283.705882,4077.818182,2435.282609
"Bratislava IV, Karlova Ves",2962.960784,2796.581395,2773.500000,3353.666667,2539.600000


In [303]:
q20 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek':'$yearweek', 'location':'$properties.Lokalita', 'state':'$properties.Stav'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}
		
  
  }
	
]))
	
df20 = pd.json_normalize(q20)\
    .rename(columns = {'_id.location':'location', '_id.state':'state', '_id.yearweek':'yearweek'})
df20 = df20[['location', 'state', 'yearweek', 'price_m2_avg']]
df20 = df20.loc[df20['location'].isin(locs),:].pivot_table(index = ['state', 'location'], columns = 'yearweek', values = 'price_m2_avg')
df20

yearweek                                                          202019  \
state                   location                                           
kompletná rekonštrukcia Bratislava I, Staré Mesto            3849.800000   
                        Bratislava II, Podunajské Biskupice  2605.666667   
                        Bratislava II, Ružinov               3040.235294   
                        Bratislava II, Vrakuňa               1802.000000   
                        Bratislava III, Nové Mesto           2957.625000   
                        Bratislava III, Rača                 3261.333333   
                        Bratislava IV, Devínska Nová Ves     2356.333333   
                        Bratislava IV, Dúbravka              2576.250000   
                        Bratislava IV, Karlova Ves           2488.000000   
                        Bratislava IV, Lamač                         NaN   
                        Bratislava V, Jarovce                        NaN   
                        Bratislava V, Petržalka              2576.700000   
pôvodný                 Bratislava I, Staré Mesto            2873.500000   
                        Bratislava II, Podunajské Biskupice  2187.000000   
                        Bratislava II, Ružinov               2985.000000   
                        Bratislava II, Vrakuňa               1743.000000   
                        Bratislava III, Nové Mesto           3616.000000   
                        Bratislava III, Rača                 2230.750000   
                        Bratislava IV, Devínska Nová Ves     2375.000000   
                        Bratislava IV, Dúbravka                      NaN   
                        Bratislava IV, Karlova Ves           4159.000000   
                        Bratislava IV, Lamač                         NaN   
                        Bratislava V, Petržalka              2483.600000   
vo výstavbe             Bratislava I, Staré Mesto                    NaN   
                        Bratislava II, Podunajské Biskupice          NaN   
                        Bratislava II, Ružinov               3218.500000   
                        Bratislava II, Vrakuňa               1927.000000   
                        Bratislava III, Nové Mesto           3494.333333   
                        Bratislava IV, Devínska Nová Ves     2501.500000   
                        Bratislava IV, Dúbravka                      NaN   
                        Bratislava IV, Karlova Ves           2579.000000   
                        Bratislava V, Jarovce                        NaN   
                        Bratislava V, Petržalka              3238.500000   
čiastočná rekonštrukcia Bratislava I, Staré Mesto            3497.250000   
                        Bratislava II, Podunajské Biskupice  1649.000000   
                        Bratislava II, Ružinov               2386.750000   
                        Bratislava II, Vrakuňa               1730.333333   
                        Bratislava III, Nové Mesto           2651.800000   
                        Bratislava III, Rača                 2895.000000   
                        Bratislava IV, Devínska Nová Ves     2052.500000   
                        Bratislava IV, Dúbravka              2475.333333   
                        Bratislava IV, Karlova Ves           2263.333333   
                        Bratislava IV, Lamač                         NaN   
                        Bratislava V, Petržalka              2574.533333   

yearweek                                                          202020  \
state                   location                                           
kompletná rekonštrukcia Bratislava I, Staré Mesto            3942.500000   
                        Bratislava II, Podunajské Biskupice  2341.000000   
                        Bratislava II, Ružinov               2943.416667   
                        Bratislava II, Vrakuňa               2133.833333   
                        Bratislava III, Nové Me

## avg m2 price per state, location and category

In [197]:
q21 = list(adcollection_d.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
	{'$group':{'_id':{'location':'$properties.Lokalita', 'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'},
              'ads_count':{'$sum':1}}
		
  
  }
	
]))
	
df21 = pd.json_normalize(q21)\
    .rename(columns = {'_id.location':'location', '_id.state':'state', '_id.category':'category'})\
    .sort_values('price_m2_avg', ascending = False)
df21 = df21.loc[df21['location'].isin(locs),:].pivot_table(index = ['location', 'category'], columns = 'state', values = ['price_m2_avg', 'ads_count'])
df21.columns = df21.columns.swaplevel(0, 1)
df21.sort_index(axis=1, level=0, inplace=True)
df21

state                                                   kompletná rekonštrukcia  \
                                                                      ads_count   
location                            category                                      
Bratislava I, Staré Mesto           1 izbový byt                           31.0   
                                    2 izbový byt                          132.0   
                                    3 izbový byt                          137.0   
                                    4 izbový byt                           44.0   
                                    5 izbový byt a viac                    15.0   
                                    Apartmán                                1.0   
                                    Garsónka                               14.0   
                                    Mezonet                                 NaN   
Bratislava II, Podunajské Biskupice 1 izbový byt                            1.0   
                                    2 izbový byt                           29.0   
                                    3 izbový byt                           23.0   
                                    4 izbový byt                            8.0   
                                    5 izbový byt a viac                     NaN   
                                    Garsónka                                2.0   
Bratislava II, Ružinov              1 izbový byt                           28.0   
                                    2 izbový byt                          135.0   
                                    3 izbový byt                           82.0   
                                    4 izbový byt                           25.0   
                                    5 izbový byt a viac                     NaN   
                                    Dvojgarsónka                            1.0   
                                    Garsónka                               13.0   
                                    Iný byt                                 1.0   
                                    Mezonet                                 4.0   
Bratislava II, Vrakuňa              1 izbový byt                            1.0   
                                    2 izbový byt                           10.0   
                                    3 izbový byt                           11.0   
                                    4 izbový byt                           16.0   
                                    5 izbový byt a viac                     NaN   
                                    Dvojgarsónka                            1.0   
                                    Garsónka                                NaN   
Bratislava III, Nové Mesto          1 izbový byt                            7.0   
                                    2 izbový byt                           73.0   
                                    3 izbový byt                           54.0   
                                    4 izbový byt                           18.0   
                                    5 izbový byt a viac                     NaN   
                                    Apartmán                                NaN   
                                    Garsónka                                1.0   
                                    Iný byt                                 NaN   
                                    Mezonet                                 1.0   
Bratislava III, Rača                1 izbový byt                           13.0   
                                    2 izbový byt                           30.0   
                                    3 izbový byt                            5.0   
                                    4 izbový byt                            2.0   
                                    5 izbový byt a viac                     NaN   
                                    Garsónka                                1.0   
Bratislava IV, D

In [307]:
q21 = list(adcollection.aggregate([
    {'$match':{'scraped timestamp': {'$lt': end, '$gte': start}}},
    {'$project':{'yearweek':{'$concat':[{'$toString':{'$year':'$scraped timestamp'}},
       {'$toString':{'$week':'$scraped timestamp'}}]
        },
        'properties':1}
    },
	{'$group':{'_id':{'yearweek': '$yearweek', 'location':'$properties.Lokalita', 'state':'$properties.Stav', 'category':'$properties.Kategória'},
			'price_m2_avg':{'$avg':'$properties.Cena za meter'}}  
  }
	
]))
	
df21 = pd.json_normalize(q21)\
    .rename(columns = {'_id.location':'location', '_id.state':'state', '_id.category':'category', '_id.yearweek':'yearweek'})
df21 = df21.loc[df21['location'].isin(locs),:].pivot_table(index = ['category', 'state', 'location'], columns = 'yearweek', values = 'price_m2_avg')
df21

yearweek                                                                              202019  \
category            state                   location                                           
1 izbový byt        kompletná rekonštrukcia Bratislava I, Staré Mesto            4718.000000   
                                            Bratislava II, Podunajské Biskupice          NaN   
                                            Bratislava II, Ružinov               3674.000000   
                                            Bratislava II, Vrakuňa                       NaN   
                                            Bratislava III, Nové Mesto                   NaN   
                                            Bratislava III, Rača                 3625.000000   
                                            Bratislava IV, Devínska Nová Ves             NaN   
                                            Bratislava IV, Dúbravka                      NaN   
                                            Bratislava IV, Karlova Ves                   NaN   
                                            Bratislava IV, Lamač                         NaN   
                                            Bratislava V, Petržalka              3054.000000   
                    pôvodný                 Bratislava I, Staré Mesto                    NaN   
                                            Bratislava II, Podunajské Biskupice          NaN   
                                            Bratislava II, Ružinov                       NaN   
                                            Bratislava III, Nové Mesto                   NaN   
                                            Bratislava IV, Devínska Nová Ves     2375.000000   
                                            Bratislava IV, Dúbravka                      NaN   
                                            Bratislava IV, Karlova Ves                   NaN   
                                            Bratislava IV, Lamač                         NaN   
                                            Bratislava V, Petržalka              3054.000000   
                    vo výstavbe             Bratislava II, Ružinov                       NaN   
                                            Bratislava III, Nové Mesto           3409.000000   
                    čiastočná rekonštrukcia Bratislava I, Staré Mesto            3744.500000   
                                            Bratislava II, Podunajské Biskupice          NaN   
                                            Bratislava II, Ružinov                       NaN   
                                            Bratislava II, Vrakuňa                       NaN   
                                            Bratislava III, Nové Mesto                   NaN   
                                            Bratislava III, Rača                 2895.000000   
                                            Bratislava IV, Devínska Nová Ves     2195.000000   
                                            Bratislava IV, Dúbravka                      NaN   
                                            Bratislava IV, Karlova Ves                   NaN   
                                            Bratislava IV, Lamač                         NaN   
                                            Bratislava V, Petržalka              2961.000000   
2 izbový byt        kompletná rekonštrukcia Bratislava I, Staré Mesto            4229.000000   
                                            Bratislava II, Podunajské Biskupice  2605.666667   
                                            Bratislava II, Ružinov               3001.000000   
                                            Bratislava II, Vrakuňa               1461.000000   
                                            Bratislava III, Nové Mesto           2961.600000   
                                            Bratislava III, Rača                 3079.500000   
                                            Bratislava IV, Devín

# TODO

- avg price section ongoing filter out duplicate ads + date filter DONE
- replace the extreme outliers in Cena & Cena za meter and set to global avg DONE
- replace avg timeseries dataframes with charts
- rounding + conditional formatting of tables
- every section add timeseries per week, data and chart
- another section DISTRIBUTION with charts, percentiles overall and as timeseries